In [ ]:
!pip install transformers datasets torch sklearn -q

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from huggingface_hub import login
login()

In [ ]:
!pip install -U transformers


In [ ]:
pip install torch transformers scikit-learn pandas


In [ ]:
import transformers
print(transformers.__version__)


4.57.1


In [ ]:
# ===============================
# LIGHTWEIGHT MENTALBERT SENTIMENT CLASSIFIER
# ===============================

import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

# 1️⃣ Load dataset
df = pd.read_csv("Cleaned_Combined_Data.csv")

TEXT_COL = "statement"
LABEL_COL = "status"

# Encode string labels to integers
df[LABEL_COL] = df[LABEL_COL].astype('category')
df['label_id'] = df[LABEL_COL].cat.codes
label_mapping = dict(enumerate(df[LABEL_COL].cat.categories))
print("✅ Label mapping:", label_mapping)

# Split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df[TEXT_COL].tolist(),
    df['label_id'].tolist(),
    test_size=0.2,
    random_state=42
)

# 2️⃣ Load tokenizer
model_name = "mental/mental-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3️⃣ Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Prepare datasets
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

# 4️⃣ Load model
num_labels = len(label_mapping)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# ✅ Freeze lower layers for speed
for name, param in model.named_parameters():
    if not name.startswith("classifier") and not name.startswith("bert.encoder.layer.11"):
        param.requires_grad = False

# 5️⃣ Training Configuration (insert this BEFORE creating Trainer)

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",      # Evaluate after each epoch
    save_strategy="epoch",            # Save checkpoints after each epoch
    learning_rate=3e-5,               # Default learning rate for BERT
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,               # You can later test 3, 5, or 8 for experiments
    weight_decay=0.1,                 # Helps prevent overfitting
    logging_dir="./logs",
    logging_steps=100,
    fp16=torch.cuda.is_available(),   # Enable half precision if GPU supports it
    load_best_model_at_end=True,      # Save best model automatically
    metric_for_best_model="f1",       # Evaluate model based on F1-score
    report_to="none"                  # Disable W&B or other trackers
)


# 6️⃣ Define metrics
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    acc = accuracy_score(p.label_ids, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# 7️⃣ Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

# 8️⃣ Evaluate model
metrics = trainer.evaluate()
print("\n📊 Evaluation Metrics:")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Save fine-tuned model
model.save_pretrained("./mentalbert-light-sentiment")
tokenizer.save_pretrained("./mentalbert-light-sentiment")

print("\n✅ Training complete! Model saved to ./mentalbert-light-sentiment")

# 9️⃣ User Input for Prediction
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("\n💬 Type a sentence to analyze (type 'quit' to exit)\n")

while True:
    text = input("Enter a sentence: ")
    if text.lower() == "quit":
        print("👋 Exiting.")
        break

    encoding = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=64
    ).to(device)

    with torch.no_grad():
        outputs = model(**encoding)
        preds = torch.softmax(outputs.logits, dim=-1)
        pred_label = torch.argmax(preds, dim=1).item()
        confidence = preds[0][pred_label].item()

    label_name = label_mapping[pred_label]
    print(f"🧠 Prediction: {label_name} ({confidence:.2%} confidence)\n")

✅ Label mapping: {0: 'Normal', 1: 'Suicidal'}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-71641494.py:106: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.075500,0.100773,0.969809,0.969791,0.969809,0.969795
2,0.095800,0.095257,0.973884,0.973874,0.973884,0.973877



📊 Evaluation Metrics:
eval_loss: 0.0953
eval_accuracy: 0.9739
eval_precision: 0.9739
eval_recall: 0.9739
eval_f1: 0.9739
eval_runtime: 11.9875
eval_samples_per_second: 450.3850
eval_steps_per_second: 56.3080
epoch: 2.0000

✅ Training complete! Model saved to ./mentalbert-light-sentiment

💬 Type a sentence to analyze (type 'quit' to exit)

Enter a sentence: I am so Sad
🧠 Prediction: Normal (71.15% confidence)

Enter a sentence: I felt so Lost lately
🧠 Prediction: Normal (99.83% confidence)

Enter a sentence: I am feeling anxiety
🧠 Prediction: Normal (98.31% confidence)

Enter a sentence: i felt alone
🧠 Prediction: Normal (98.30% confidence)

Enter a sentence: I want to disappear
🧠 Prediction: Suicidal (99.60% confidence)

Enter a sentence: I want to die
🧠 Prediction: Suicidal (99.91% confidence)

Enter a sentence: I don't want to be here
🧠 Prediction: Normal (99.78% confidence)

Enter a sentence: I am angry to everyone
🧠 Prediction: Suicidal (97.82% confidence)

Enter a sentence: I am 

In [ ]:
# ===============================
# MENTALBERT SENTIMENT CLASSIFIER (with Fine-Tuning)
# ===============================

import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 1️⃣ Load dataset
df = pd.read_csv("Cleaned_Combined_Data.csv")

TEXT_COL = "statement"
LABEL_COL = "status"

# Encode string labels to integers
df[LABEL_COL] = df[LABEL_COL].astype('category')
df['label_id'] = df[LABEL_COL].cat.codes
label_mapping = dict(enumerate(df[LABEL_COL].cat.categories))
print("✅ Label mapping:", label_mapping)

# Split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df[TEXT_COL].tolist(),
    df['label_id'].tolist(),
    test_size=0.2,
    random_state=42
)

# 2️⃣ Load tokenizer
model_name = "mental/mental-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3️⃣ Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Prepare datasets
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

# 4️⃣ Load model
num_labels = len(label_mapping)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# ✅ Freeze lower layers for lightweight fine-tuning
for name, param in model.named_parameters():
    if not name.startswith("classifier") and not name.startswith("bert.encoder.layer.11"):
        param.requires_grad = False

# 5️⃣ Fine-Tuning Hyperparameters
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",       # Evaluate every epoch
    save_strategy="epoch",             # Save checkpoint every epoch
    learning_rate=3e-5,                # Good baseline for BERT models
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,                # You can assign 2–5 for your members to test
    weight_decay=0.1,                 # Regularization to avoid overfitting
    logging_dir="./logs",
    logging_steps=100,
    fp16=torch.cuda.is_available(),    # Mixed precision if GPU is available
    load_best_model_at_end=True,       # Keep the best checkpoint
    metric_for_best_model="f1",        # Evaluate best model by F1-score
    report_to="none"                   # Disable W&B/MLflow
)

# 6️⃣ Metrics Function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    acc = accuracy_score(p.label_ids, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        p.label_ids, preds, average='weighted'
    )
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# 7️⃣ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 8️⃣ Train & Evaluate
trainer.train()
metrics = trainer.evaluate()

print("\n📊 Evaluation Metrics:")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# 9️⃣ Save Model
model.save_pretrained("./mentalbert-light-sentiment")
tokenizer.save_pretrained("./mentalbert-light-sentiment")
print("\n✅ Training complete! Model saved to ./mentalbert-light-sentiment")

# 🔟 User Input for Prediction
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("\n💬 Type a sentence to analyze (type 'quit' to exit)\n")

while True:
    text = input("Enter a sentence: ")
    if text.lower() == "quit":
        print("👋 Exiting.")
        break

    encoding = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=64
    ).to(device)

    with torch.no_grad():
        outputs = model(**encoding)
        preds = torch.softmax(outputs.logits, dim=-1)
        pred_label = torch.argmax(preds, dim=1).item()
        confidence = preds[0][pred_label].item()

    label_name = label_mapping[pred_label]
    print(f"🧠 Prediction: {label_name} ({confidence:.2%} confidence)\n")

✅ Label mapping: {0: 'Normal', 1: 'Suicidal'}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4185149604.py:104: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.075500,0.100773,0.969809,0.969791,0.969809,0.969795
2,0.095800,0.095257,0.973884,0.973874,0.973884,0.973877



📊 Evaluation Metrics:
eval_loss: 0.0953
eval_accuracy: 0.9739
eval_precision: 0.9739
eval_recall: 0.9739
eval_f1: 0.9739
eval_runtime: 12.2725
eval_samples_per_second: 439.9280
eval_steps_per_second: 55.0010
epoch: 2.0000

✅ Training complete! Model saved to ./mentalbert-light-sentiment

💬 Type a sentence to analyze (type 'quit' to exit)

Enter a sentence: I had such a productive day at work and I feel proud of myself.
🧠 Prediction: Normal (99.89% confidence)

Enter a sentence: Can’t wait to hang out with my friends this weekend!
🧠 Prediction: Normal (99.93% confidence)

Enter a sentence: I’m a bit tired today, but overall things are going fine.
🧠 Prediction: Normal (99.94% confidence)

Enter a sentence: I just finished watching my favorite show, it made me laugh so much.
🧠 Prediction: Normal (99.93% confidence)

Enter a sentence: I don’t see any reason to keep living anymore.
🧠 Prediction: Normal (99.76% confidence)

Enter a sentence: I’m trying to focus more on my health and stay po